In [1]:
%use coroutines

In [17]:
// Java
import java.util.Base64
import javax.crypto.Cipher
import javax.crypto.SecretKeyFactory
import javax.crypto.spec.PBEKeySpec
import javax.crypto.spec.PBEParameterSpec

// Kotlin
import kotlin.random.Random
import kotlin.properties.Delegates

In [3]:
var str = "Hello !!!ðŸ˜Žworld @1233*@@():)ðŸ‘ŒðŸ‘ŒðŸ‘Œ ] ["

// [^ ... ]  Match any character that is not in the set.
// Matches a character in the range "A" to "Z" (char code 65 to 90). Case sensitive.
// Matches a character in the range "a" to "z" (char code 97 to 122). Case sensitive.
// Matches a character in the range "0" to "9" (char code 48 to 57). Case sensitive.
// Matches a SPACE character (char code 32). 

val re = "[^A-Za-z0-9 ]".toRegex()
str = re.replace(str, "")

println(str)

Hello world 1233  


In [4]:
var c: Char

c = 'A'
while (c <= 'z') {
    print("$c ")
    ++c
}

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z 

In [5]:
val bytes = byteArrayOf(75, 111, 116, 108, 105, 110)

val string = String(bytes)

println(string)

Kotlin


In [6]:
val chars = ('a'..'z') + ('A'..'Z') + ('0'..'9')
fun randomID(length: Int = 64): String = List(length) { chars.random() }.joinToString("")

println(chars);
println(randomID())

[a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
n0Yk0mzF2AMIGLznFaOAnMBSKV0MML9RBBqfR657ePIWS1vOaGpdFjnW5PXsRy8F


In [7]:
// https://stackoverflow.com/questions/54143361/what-is-pbewithmd5anddes
// https://gist.github.com/rohitshampur/da5f79c34260150aafc1
// https://www.bezkoder.com/kotlin-base64/

val secretCode = randomID().toCharArray()
val salt = List(8) { chars.random() }.joinToString("").toByteArray() // Salt must be 8 bytes long

fun encrypt(value: String?): String {
    val strBytes = value?.toByteArray()
    val keyFactory = SecretKeyFactory.getInstance("PBEWithMD5AndDES")
    val key = keyFactory.generateSecret(PBEKeySpec(secretCode))
    val cipher = Cipher.getInstance("PBEWithMD5AndDES").also {
        it.init(
            Cipher.ENCRYPT_MODE, key,
            PBEParameterSpec(
                salt, 20
            )
        )
    }
    // println(String(cipher.doFinal(strBytes)))
    // println()
    return Base64.getEncoder().withoutPadding().encodeToString(cipher.doFinal(strBytes))
}

fun decode(value: String?): ByteArray {
    val decodedBytes = Base64.getDecoder().decode(value)
    return decodedBytes
}

fun decrypt(value: String?): String {
    val strBytes = decode(value)
    val keyFactory = SecretKeyFactory.getInstance("PBEWithMD5AndDES")
    val key = keyFactory.generateSecret(PBEKeySpec(secretCode))
    val cipher = Cipher.getInstance("PBEWithMD5AndDES")
    cipher.init(
        Cipher.DECRYPT_MODE, key,
        PBEParameterSpec(
            salt, 20
        )
    )
    return String(cipher.doFinal(strBytes), Charsets.UTF_8)
}

val stringToEncode = randomID()
val encryptedString = encrypt(stringToEncode)
val decryptedString = decrypt(encryptedString)

println("initial value: $stringToEncode")
println()
println("salt: ${String(salt)}")
println("secret code: ${String(secretCode)}")
println()
println("encrypted: $encryptedString")
// println(String(decode(encryptedString)))
println()
println("decrypted: $decryptedString")
println()
println("is value equal: ${stringToEncode == decryptedString}")

initial value: 8hzkacRk6DveyRzPsgMQpFTHMLDs9MQNKNtjp6qD0iV1TwjjJQSRpiSrmtSB1Xkm

salt: oPfNYnr2
secret code: xNkQXElJAnk9PKqVYhbvtPHO6LP7rB9CTsGwvFrde90YZ5yJs9k3y0ghVhX4piX6

encrypted: slRiOAA5/thBYaPhQ/kYE66LK3ctWGwPd5hYROOTuh/w8QP3DBwy+3EFwWHFoa7ia6dB67vPj3vq5MSHUIgcVaeHVeA8EDFZ

decrypted: 8hzkacRk6DveyRzPsgMQpFTHMLDs9MQNKNtjp6qD0iV1TwjjJQSRpiSrmtSB1Xkm

is value equal: true


In [8]:
import java.time.Instant

inline fun time(action: () -> Unit): Long {
    val start = Instant.now().toEpochMilli()
    action()
    return Instant.now().toEpochMilli() - start
}

suspend fun doWork() = coroutineScope {
    launch {
        val stringToEncode = randomID(64)
        val encryptedString = encrypt(stringToEncode)
        val decryptedString = decrypt(encryptedString)
        
        println("encrypted: $encryptedString")
        println()
        println("decrypted: $decryptedString")
    }
}

fun main() = runBlocking {
    val t = time {
        doWork()
    }
    println("Done $t ms")
}

main()

encrypted: iqZBDm/WhafX8ANrpKY5whdJM6hw7ZKwEqJEE/X5ywqZ2Z3Ka3sbpIqsMsB1r06e4IqbfX1da6rDnfVfPJSwXh8Sbljkk3pe

decrypted: 1DPXYMIBgAHj5wKHKvukspugkxY2tAcKfbS4I2zU4NMVUcieGV5d6F5fmF4iMbQu
Done 35 ms


https://cyware.com/news/exploring-the-differences-between-symmetric-and-asymmetric-encryption-8de86e8a

The basic difference between these two types of encryption is that symmetric encryption uses one key for both encryption and decryption, and the asymmetric encryption uses public key for encryption and a private key for decryption.

https://securityboulevard.com/2020/06/when-to-use-symmetric-vs-asymmetric-encryption-keyfactor/

Common Use Cases for Symmetric Cryptography <br/>
Symmetric cryptography typically gets used when speed is the priority over increased security, keeping in mind that encrypting a message still offers a high level of security. Some of the most common use cases for symmetric cryptography include: <br/>

Banking: Encrypting credit card information or other personally identifiable information (PII) required for transactions <br/>
Data storage: Encrypting data stored on a device when that data is not being transferred  <br/>

Common Use Cases for Asymmetric Cryptography <br/>
Asymmetric cryptography typically gets used when increased security is the priority over speed and when identity verification is required, as the latter is not something symmetric cryptography supports. Some of the most common use cases for asymmetric cryptography include: <br/>

Digital signatures: Confirming identity for someone to sign a document <br/>
Blockchain: Confirming identity to authorize transactions for cryptocurrency <br/>
Public key infrastructure (PKI): Governing encryption keys through the issuance and management of digital certificates <br/>

In [29]:
import java.security.MessageDigest
import java.security.DigestException
import java.lang.CloneNotSupportedException

https://www.geeksforgeeks.org/difference-between-sha1-and-sha256/

https://docs.oracle.com/javase/7/docs/api/java/security/MessageDigest.html

This MessageDigest class provides applications the functionality of a message digest algorithm, such as SHA-1 or SHA-256. Message digests are secure one-way hash functions that take arbitrary-sized data and output a fixed-length hash value.
A MessageDigest object starts out initialized. The data is processed through it using the update methods. At any point reset can be called to reset the digest. Once all the data to be updated has been updated, one of the digest methods should be called to complete the hash computation.

The digest method can be called once for a given number of updates. After digest has been called, the MessageDigest object is reset to its initialized state.

Implementations are free to implement the Cloneable interface. Client applications can test cloneability by attempting cloning and catching the CloneNotSupportedException:

In [31]:
// https://stackoverflow.com/questions/18438946/format-specifier-02x

fun String.passwordHash(): String {
    val digest = MessageDigest.getInstance("SHA-1")
    val result = digest.digest(toByteArray(Charsets.UTF_8))
    val sb = StringBuilder()
    for (b in result) {
        sb.append(String.format("%02X", b))
    }
    return sb.toString()
}

"Hello World".passwordHash()


0A4D55A8D778E5022FAB701977C5D840BBC486D0

https://docs.oracle.com/javase/7/docs/api/javax/crypto/KeyGenerator.html

This class provides the functionality of a secret (symmetric) key generator. <br/>
Key generators are constructed using one of the getInstance class methods of this class.

KeyGenerator objects are reusable, i.e., after a key has been generated, the same KeyGenerator object can be re-used to generate further keys. <br/>

There are two ways to generate a key: in an algorithm-independent manner, and in an algorithm-specific manner. The only difference between the two is the initialization of the object: <br/>

Algorithm-Independent Initialization <br/>
All key generators share the concepts of a keysize and a source of randomness. There is an init method in this KeyGenerator class that takes these two universally shared types of arguments. There is also one that takes just a keysize argument, and uses the SecureRandom implementation of the highest-priority installed provider as the source of randomness (or a system-provided source of randomness if none of the installed providers supply a SecureRandom implementation), and one that takes just a source of randomness. <br/>

Since no other parameters are specified when you call the above algorithm-independent init methods, it is up to the provider what to do about the algorithm-specific parameters (if any) to be associated with each of the keys. <br/>

Algorithm-Specific Initialization <br/>
For situations where a set of algorithm-specific parameters already exists, there are two init methods that have an AlgorithmParameterSpec argument. One also has a SecureRandom argument, while the other uses the SecureRandom implementation of the highest-priority installed provider as the source of randomness (or a system-provided source of randomness if none of the installed providers supply a SecureRandom implementation). <br/>

In case the client does not explicitly initialize the KeyGenerator (via a call to an init method), each provider must supply (and document) a default initialization. <br/>

Every implementation of the Java platform is required to support the following standard KeyGenerator algorithms with the keysizes in parentheses: <br/>

AES (128) <br/>
DES (56) <br/>
DESede (168) <br/>
HmacSHA1 <br/>
HmacSHA256 <br/>
These algorithms are described in the KeyGenerator section of the Java Cryptography Architecture Standard Algorithm Name Documentation. Consult the release documentation for your implementation to see if any other algorithms are supported. <br/>

https://docs.oracle.com/javase/7/docs/technotes/guides/security/StandardNames.html
https://stackoverflow.com/questions/37231211/cannot-decrypt-a-file-that-i-encrypted-using-pbewithhmacsha256andaes-128

PBEWithHmacSHA256AndAES_128

In [131]:
import javax.crypto.KeyGenerator
import javax.crypto.SecretKey

import java.io.ByteArrayOutputStream
import java.io.ObjectOutputStream

In [88]:
fun generateKey(): SecretKey {
    return KeyGenerator.getInstance("AES").apply {
        init(256)
    }.generateKey()
}

fun SecretKey.secretKeyToString(): String {
    val baos = ByteArrayOutputStream()
    val oos = ObjectOutputStream(baos)
    oos.writeObject(this)
    return Base64.getEncoder().withoutPadding().encodeToString(baos.toByteArray())
}

val secretKey = generateKey()

println("Secret key: ${secretKey.secretKeyToString()}")

Secret key: rO0ABXNyAB9qYXZheC5jcnlwdG8uc3BlYy5TZWNyZXRLZXlTcGVjW0cLZuIwYU0CAAJMAAlhbGdvcml0aG10ABJMamF2YS9sYW5nL1N0cmluZztbAANrZXl0AAJbQnhwdAADQUVTdXIAAltCrPMX+AYIVOACAAB4cAAAACAtNgD3rvIHg3as2s/ymdypZa5iM13yeaK3oJd+LaD/mg


In [127]:
fun String.encrypt(key: SecretKey): String {
    val cipher = Cipher.getInstance("AES/CBC/PKCS5PADDING").apply {
        init(Cipher.ENCRYPT_MODE, key)
    }
    
    val initVector = cipher.iv
    val cipherText = cipher.doFinal(toByteArray(Charsets.UTF_8))
    return Base64.getEncoder().withoutPadding().encodeToString(cipherText)
}

val encrypted = "Indra Mahkota".encrypt(secretKey)

println(encrypted)

4ALLymCX3TucJM2Y+d3glQ


I found out the AES/CBC/PKCS5Padding isn't secure and suggesting to move on to AES/GCM/NoPadding.
https://stackoverflow.com/questions/57668541/galois-counter-mode-gcm-nopadding-not-used

In [132]:
fun String.decrypt(key: SecretKey, initVector: IvParameterSpec): String {
    val cipher = Cipher.getInstance("AES/CBC/PKCS5PADDING").apply {
        init(Cipher.DECRYPT_MODE, key, initVector)
    }
    val cipherText = cipher.doFinal(dataToDecrypt)
    return Base64.getEncoder().withoutPadding().encodeToString(cipherText)
}

val decrypted = encrypted.decrypt(secretKey)

println(decrypted)

Line_1238.jupyter-kts (1:48 - 63) Unresolved reference: IvParameterSpec
Line_1238.jupyter-kts (3:9 - 13) Overload resolution ambiguity: 
public final fun init(p0: Int, p1: Key!, p2: AlgorithmParameters!): Unit defined in javax.crypto.Cipher
public final fun init(p0: Int, p1: Key!, p2: SecureRandom!): Unit defined in javax.crypto.Cipher
public final fun init(p0: Int, p1: Key!, p2: AlgorithmParameterSpec!): Unit defined in javax.crypto.Cipher
Line_1238.jupyter-kts (5:37 - 50) Unresolved reference: dataToDecrypt